In [1]:
# (ignore this) imports and utility functions

import re
import json
import toolkit

from IPython.display import clear_output, display, Markdown


def render_markdown(text):
    if text is None:
        text = ""

    pattern = r"```thinking\n(.*?)\n```"

    def replacement(match):
        content = match.group(1)
        return f"<blockquote>\n{content}\n</blockquote>\n\n"

    text = re.sub(pattern, replacement, text, flags=re.DOTALL)

    text = re.sub(r"```function_spec\b", "```json", text)
    text = re.sub(r"```function_call\b", "```json", text)
    text = re.sub(r"```function_output\b", "```json", text)

    display(Markdown(text))


def render_json(data):
    return render_markdown(f"```json\n{json.dumps(data, indent=2)}\n```\n")


def render_live(stream):
    response = ""
    render_markdown("> Loading...")

    for chunk in stream:
        response += chunk
        clear_output(wait=True)
        render_markdown(response)

    return response

In [2]:
model = "gemma3:27b"

In [3]:
await toolkit.code.create_sandbox()

In [4]:
async def run_python_code(code):
    runner = await toolkit.code.sandbox.run(code)
    output = await runner.output()
    errors = await runner.error()

    if errors:
        return {
            "error": {
                "name": "RuntimeException",
                "message": f"Function execution produced runtime errors: {errors}",
            }
        }

    return {"result": output}


async def install_pip_packages(packages):
    runner = await toolkit.code.sandbox.run(f"pip install {' '.join(packages)}")
    output = await runner.output()
    errors = await runner.error()

    if errors:
        return {
            "error": {
                "name": "RuntimeException",
                "message": f"Package installation produced runtime errors: {errors}",
            }
        }

    return {"result": output}


async def run_shell_command(command, arguments=[]):
    runner = await toolkit.code.sandbox.command.run(command, arguments)
    output = await runner.output()
    errors = await runner.error()

    if errors:
        return {
            "error": {
                "name": "RuntimeException",
                "message": f"Command execution produced runtime errors: {errors}",
            }
        }

    return {"result": output}

In [5]:
specifications = {
    "run_python_code": {
        "name": "run_python_code",
        "description": "Executes a given Python code string in a secure, sandboxed environment and returns the standard output. This is useful for simple and complex scripts that allow you to perform calculations, data manipulation, or interacting with external APIs via network requests. If you do not have access to a function, write python code for the function and run it by calling this function instead.",
        "parameters": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "A string containing the valid Python code to be executed.",
                }
            },
            "required": ["code"],
        },
        "responses": [
            {
                "type": "string",
                "description": "The standard output (stdout) generated by the successful execution of the code.",
            }
        ],
        "errors": [
            {
                "name": "Exception",
                "description": "Raised if the executed code produces any runtime errors or exceptions. The error message will contain the details from stderr.",
            }
        ],
    },
    "install_pip_packages": {
        "name": "install_pip_packages",
        "description": "Installs the provided packages using pip, in the same environment that the code provided to `run_python_code` executes. Use it to install packages if you need them.",
        "parameters": {
            "type": "object",
            "properties": {
                "packages": {
                    "type": "array",
                    "description": "The packages to install, will be passed to `pip install` as space-separated arguments.",
                    "items": {"type": "string"},
                }
            },
            "required": ["packages"],
        },
        "responses": [
            {
                "type": "string",
                "description": "The standard output (stdout) generated by the successful installation of the packages.",
            }
        ],
        "errors": [
            {
                "name": "Exception",
                "description": "Raised if the installation process produces any runtime errors or exceptions. The error message will contain the details from stderr.",
            }
        ],
    },
    "run_shell_command": {
        "name": "run_shell_command",
        "description": "Executes a command in a BASH shell, in the same environment that the python code provided to `run_python_code` executes. Use it to run arbitrary shell commands.",
        "parameters": {
            "type": "object",
            "properties": {
                "command": {
                    "type": "string",
                    "description": "The command to execute in the shell.",
                },
                "arguments": {
                    "type": "array",
                    "description": "The arguments to provide to the command.",
                    "items": {"type": "string"},
                },
            },
            "required": ["command"],
        },
        "responses": [
            {
                "type": "string",
                "description": "The standard output (stdout) generated by the successful execution of the command.",
            }
        ],
        "errors": [
            {
                "name": "Exception",
                "description": "Raised if the command produces a non zero exit code. The error message will contain the details from stderr.",
            }
        ],
    },
}

In [6]:
toolkit.functions.register(run_python_code, specifications["run_python_code"])
toolkit.functions.register(install_pip_packages, specifications["install_pip_packages"])
toolkit.functions.register(run_shell_command, specifications["run_shell_command"])

In [7]:
messages = []
prompt = toolkit.prompts.function_calling(model)
specs = toolkit.functions.specify()
task = "What's the weather in Pune? Install the requests library and use the wttr.in/<place> API to fetch it please."

In [8]:
messages.append({"role": "user", "content": prompt})
messages.append({"role": "user", "content": specs})
messages.append({"role": "user", "content": task})

In [9]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
Okay, I need to find the weather in Pune. The user wants me to use the `wttr.in/<place>` API. Since I don't have direct access to external APIs, I need to use `run_python_code` to make a request to this API. Also, the user explicitly asked me to install the `requests` library before using it, so I should do that first. I will use `install_pip_packages` to install the `requests` library, and then I will use `run_python_code` to fetch the weather.
</blockquote>



```json
{
	"id": "1",
	"function": "install_pip_packages",
	"parameters": {
		"packages": [
			"requests"
		]
	}
}
```

In [10]:
calls = toolkit.functions.parse_calls(complete_response)
outputs = await toolkit.functions.execute_calls(calls)
messages.append({"role": "user", "content": outputs})
render_markdown(outputs)

```json
{
    "id": "1",
    "result": ""
}
```


In [11]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

<blockquote>
The `install_pip_packages` function call was successful (empty result indicates success). Now I can proceed to fetch the weather using `run_python_code`. I will construct a Python script to use the `requests` library to access the `wttr.in/Pune` API and print the response.
</blockquote>



```json
{
	"id": "2",
	"function": "run_python_code",
	"parameters": {
		"code": "import requests\n\ntry:\n    response = requests.get('http://wttr.in/Pune')\n    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)\n    print(response.text)\nexcept requests.exceptions.RequestException as e:\n    print(f\"An error occurred: {e}\")"
	}
}
```

In [12]:
calls = toolkit.functions.parse_calls(complete_response)
outputs = await toolkit.functions.execute_calls(calls)
messages.append({"role": "user", "content": outputs})
render_markdown(outputs)

```json
{
    "id": "2",
    "result": "Weather report: Pune\n\n  \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain nearby\n  \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;226m+24\u001b[0m(\u001b[38;5;220m26\u001b[0m) \u00b0C\u001b[0m     \n  \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;220m16\u001b[0m km/h\u001b[0m      \n  \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \n  \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.0 mm\u001b[0m         \n                                                       \u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510                                                       \n\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2524  Thu 26 Jun \u251c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510\n\u2502            Morning           \u2502             Noon      \u2514\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2518     Evening           \u2502             Night            \u2502\n\u251c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u253c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u253c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u253c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2524\n\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502\n\u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;220m+25\u001b[0m(\u001b[38;5;220m27\u001b[0m) \u00b0C\u001b[0m     \u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;220m+26\u001b[0m(\u001b[38;5;214m28\u001b[0m) \u00b0C\u001b[0m     \u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;226m+24\u001b[0m(\u001b[38;5;220m26\u001b[0m) \u00b0C\u001b[0m     \u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;226m+23\u001b[0m(\u001b[38;5;220m25\u001b[0m) \u00b0C\u001b[0m     \u2502\n\u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;208m24\u001b[0m-\u001b[38;5;208m27\u001b[0m km/h\u001b[0m   \u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;208m24\u001b[0m-\u001b[38;5;208m27\u001b[0m km/h\u001b[0m   \u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;220m16\u001b[0m-\u001b[38;5;220m19\u001b[0m km/h\u001b[0m   \u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;226m13\u001b[0m-\u001b[38;5;220m17\u001b[0m km/h\u001b[0m   \u2502\n\u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502\n\u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.0 mm | 62%\u001b[0m   \u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.0 mm | 86%\u001b[0m   \u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.0 mm | 73%\u001b[0m   \u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.0 mm | 83%\u001b[0m   \u2502\n\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2534\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2534\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2534\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518\n                                                       \u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510                                                       \n\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2524  Fri 27 Jun \u251c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510\n\u2502            Morning           \u2502             Noon      \u2514\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2518     Evening           \u2502             Night            \u2502\n\u251c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u253c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u253c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u253c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2524\n\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502\n\u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;226m+23\u001b[0m(\u001b[38;5;220m25\u001b[0m) \u00b0C\u001b[0m     \u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;226m+23\u001b[0m(\u001b[38;5;220m25\u001b[0m) \u00b0C\u001b[0m     \u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;226m+23\u001b[0m(\u001b[38;5;220m25\u001b[0m) \u00b0C\u001b[0m     \u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;226m+22\u001b[0m(\u001b[38;5;220m25\u001b[0m) \u00b0C\u001b[0m     \u2502\n\u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;226m13\u001b[0m-\u001b[38;5;220m16\u001b[0m km/h\u001b[0m   \u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;190m12\u001b[0m-\u001b[38;5;226m14\u001b[0m km/h\u001b[0m   \u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;190m10\u001b[0m-\u001b[38;5;190m12\u001b[0m km/h\u001b[0m   \u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;154m9\u001b[0m-\u001b[38;5;190m11\u001b[0m km/h\u001b[0m    \u2502\n\u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502\n\u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.0 mm | 85%\u001b[0m   \u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.1 mm | 100%\u001b[0m  \u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.0 mm | 61%\u001b[0m   \u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.0 mm | 74%\u001b[0m   \u2502\n\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2534\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2534\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2534\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518\n                                                       \u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510                                                       \n\u250c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2524  Sat 28 Jun \u251c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2510\n\u2502            Morning           \u2502             Noon      \u2514\u2500\u2500\u2500\u2500\u2500\u2500\u252c\u2500\u2500\u2500\u2500\u2500\u2500\u2518     Evening           \u2502             Night            \u2502\n\u251c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u253c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u253c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u253c\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2524\n\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502 \u001b[38;5;226m _`/\"\"\u001b[38;5;250m.-.    \u001b[0m Patchy rain ne\u2026\u2502\n\u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;226m+23\u001b[0m(\u001b[38;5;220m25\u001b[0m) \u00b0C\u001b[0m     \u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;220m+26\u001b[0m(\u001b[38;5;220m27\u001b[0m) \u00b0C\u001b[0m     \u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;226m+24\u001b[0m(\u001b[38;5;220m26\u001b[0m) \u00b0C\u001b[0m     \u2502 \u001b[38;5;226m  ,\\_\u001b[38;5;250m(   ).  \u001b[0m \u001b[38;5;226m+23\u001b[0m(\u001b[38;5;220m25\u001b[0m) \u00b0C\u001b[0m     \u2502\n\u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;220m18\u001b[0m-\u001b[38;5;214m21\u001b[0m km/h\u001b[0m   \u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;208m24\u001b[0m-\u001b[38;5;202m28\u001b[0m km/h\u001b[0m   \u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;214m20\u001b[0m-\u001b[38;5;208m24\u001b[0m km/h\u001b[0m   \u2502 \u001b[38;5;226m   /\u001b[38;5;250m(___(__) \u001b[0m \u001b[1m\u2197\u001b[0m \u001b[38;5;220m17\u001b[0m-\u001b[38;5;214m20\u001b[0m km/h\u001b[0m   \u2502\n\u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502 \u001b[38;5;111m     \u2018 \u2018 \u2018 \u2018 \u001b[0m 10 km\u001b[0m          \u2502\n\u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.0 mm | 71%\u001b[0m   \u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.1 mm | 100%\u001b[0m  \u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.1 mm | 100%\u001b[0m  \u2502 \u001b[38;5;111m    \u2018 \u2018 \u2018 \u2018  \u001b[0m 0.1 mm | 100%\u001b[0m  \u2502\n\u2514\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2534\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2534\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2534\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2500\u2518\nLocation: Pune, Maharashtra, 411001, India [18.5203062,73.8543185]\n\nFollow \u001b[46m\u001b[30m@igor_chubin\u001b[0m for wttr.in updates"
}
```


In [13]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

This is a beautifully formatted weather report presented as a text-based UI. Here's a breakdown of the information and its visual elements:

**Overall Structure:**

*   **Framing:** The report is enclosed within a border created using Unicode box-drawing characters (like `\u251c`, `\u2514`, `\u2500`, `\u253c`). This gives it a visual "card" appearance.
*   **Multi-Section:** It's divided into several sections using borders, effectively creating rows and columns.
*   **Time Slots:** The main sections clearly delineate time slots: "Morning," "Noon," "Evening," and "Night."
*   **Date:** The date "Sat 28 Jun" is prominently displayed.
*   **Location:** Provides the location "Pune, Maharashtra, 411001, India" and geographical coordinates.

**Information Displayed:**

*   **Weather Icons:**  Uses Unicode characters to represent weather conditions (e.g., `_'/"".-.` representing a cloud or rain).
*   **Temperature:** Shows both high and low temperatures in Celsius (e.g., `+23(+25)`).
*   **Wind Speed:** Displays wind speed in km/h (e.g., `18-21 km/h`).
*   **Humidity:**  Shows humidity level (e.g., `0.0 mm | 71%`).
*   **Precipitation:**  Indicates precipitation amount, if any.
*   **Sun/Moon Information:** Although not explicitly stated, the time slots suggest the report considers daytime and nighttime conditions.

**Color & Styling:**

*   **ANSI Escape Codes:** The report utilizes ANSI escape codes (e.g., `\u001b[46m\u001b[30m`) for color and styling:
    *   `\u001b[46m`: Sets the background color to cyan.
    *   `\u001b[30m`: Sets the text color to black.
    *   `\u001b[0m`: Resets the colors to default.  This is used to end the colored text.

**Specific Notes**

*   **Unicode Box-Drawing:** The report expertly leverages Unicode box-drawing characters to create a visually appealing and structured layout, even within a text-only environment.
*   **Clear and Concise:** The information is presented in a clear and concise manner.
*   **Social Media Link:** Includes a link to the developer's Twitter account for updates.
*   **Location Details:** Latitude and longitude coordinates are provided for precise location.

**How to View Correctly:**

To view this report correctly, you need a terminal or text editor that supports Unicode characters and ANSI escape codes.  Most modern terminals (like those on Linux, macOS, and newer versions of Windows) will render this report as intended.

This is a well-designed and aesthetically pleasing way to present weather information in a terminal-based application. It's a creative use of text-based UI elements.


In [14]:
await toolkit.code.sandbox.stop()